In [1]:
from simtk.openmm import app, unit
from simtk import openmm
import numpy as np
import mdtraj as md

In [2]:
protein_filename = "../../input/1brs_barnase_renumbered.pdb"
ligand_input = "../../input/1brs_barstar_renumbered.pdb"

In [3]:
# First thing to do is load the apo protein
protein_pdbfile = open(protein_filename, 'r')
protein_pdb = app.PDBFile(protein_pdbfile)
protein_pdbfile.close()
protein_positions, protein_topology, protein_md_topology = protein_pdb.positions, protein_pdb.topology, md.Topology.from_openmm(protein_pdb.topology)
protein_topology = protein_md_topology.to_openmm()
protein_n_atoms = protein_md_topology.n_atoms

# Load the ligand
ligand_pdbfile = open(ligand_input, 'r')
ligand_pdb = app.PDBFile(ligand_pdbfile)
ligand_pdbfile.close()
ligand_positions, ligand_topology, ligand_md_topology = ligand_pdb.positions, ligand_pdb.topology, md.Topology.from_openmm(
    ligand_pdb.topology)
ligand_n_atoms = ligand_md_topology.n_atoms

# Now create a complex
complex_md_topology = protein_md_topology.join(ligand_md_topology)
complex_topology = complex_md_topology.to_openmm()
complex_positions = unit.Quantity(np.zeros([protein_n_atoms + ligand_n_atoms, 3]), unit=unit.nanometers)
complex_positions[:protein_n_atoms, :] = protein_positions
complex_positions[protein_n_atoms:, :] = ligand_positions

# Convert positions back to openmm vec3 objects
complex_positions_vec3 = []
for position in complex_positions:
    complex_positions_vec3.append(openmm.Vec3(*position.value_in_unit_system(unit.md_unit_system)))
complex_positions_vec3 = unit.Quantity(value=complex_positions_vec3, unit=unit.nanometer)

In [4]:
# Load the ffxmls
ffxml = ['amber14/protein.ff14SB.xml', "amber14/tip3p.xml"]
ff = app.ForceField(*ffxml)

In [5]:
modeller = app.Modeller(complex_topology, complex_positions)
modeller.addSolvent(ff, padding=0.9 * unit.nanometers, ionicStrength=0.15 * unit.molar)
app.PDBFile.writeFile(modeller.topology, modeller.positions, open("complex_np.pdb", "w"), keepIds=True)

In [6]:
modeller = app.Modeller(complex_topology, complex_positions_vec3)
modeller.addSolvent(ff, padding=0.9 * unit.nanometers, ionicStrength=0.15 * unit.molar)
app.PDBFile.writeFile(modeller.topology, modeller.positions, open("complex_vec3.pdb", "w"), keepIds=True)

array([[ 1.4878,  5.0324,  2.7118],
       [ 1.5711,  4.9593,  2.6066],
       [ 1.5422,  4.9701,  2.4876],
       ...,
       [ 4.5191,  3.48  , -1.0616],
       [ 4.582 ,  3.4472, -0.9905],
       [ 4.4247,  3.4796, -1.0278]])

In [13]:
modeller = app.Modeller(complex_topology, np.array(complex_positions_vec3.value_in_unit_system(unit.md_unit_system)))
modeller.addSolvent(ff, padding=0.9 * unit.nanometers, ionicStrength=0.15 * unit.molar)
app.PDBFile.writeFile(modeller.topology, modeller.positions, open("complex_vec3_np.pdb", "w"), keepIds=True)